In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

base_path = "/content/drive/MyDrive/Colab Notebooks/platform_subscribe/"

train = pd.read_csv(base_path + "train.csv")
test = pd.read_csv(base_path + "test.csv")
submission = pd.read_csv(base_path + "sample_submission.csv")

In [3]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.1 MB/s eta 0:00:00


In [8]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.5 MB/s eta 0:00:00


In [9]:
# Optuna Libraries
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 평가 지표
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

evulation_metric = f1_score

### optuna+second_best = ['average_time_per_learning_session', 'recent_learning_achievement'] + data_scaling + catboost

In [10]:
second_best = ['average_time_per_learning_session', 'recent_learning_achievement']
third_best = ['average_time_per_learning_session',
   'recent_learning_achievement',
   'average_login_time',
   'subscription_duration',
   'recent_login_time',
   'monthly_active_learning_days',
   'total_completed_courses',
   'abandoned_learning_sessions',
   'payment_pattern',
   'customer_inquiry_history']

In [11]:
X=train[second_best]
y=train.target
X

,average_time_per_learning_session,recent_learning_achievement
0,8.427187,68.360455
1,72.646087,97.567322
2,21.774492,94.358763
3,42.659066,70.153228
4,30.744287,81.917908
...,...,...
9995,84.053558,64.966803
9996,45.464833,82.750244
9997,127.302411,81.567839
9998,5.297234,89.885656


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25, random_state=42)

In [24]:
# random sampler
sampler = TPESampler(seed=10)

# define function
def objective(trial):

    cbrm_param = {
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }

    # Generate model
    model_cbclf = CatBoostClassifier(**cbrm_param)
    model_cbclf = model_cbclf.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                           verbose=0, early_stopping_rounds=25)
    pred_val_cbclf=model_cbclf.predict(X_val)
    evulation_metric_val_cb = evulation_metric(y_val, pred_val_cbclf)
    return evulation_metric_val_cb

In [25]:
optuna_cbrm = optuna.create_study(direction="maximize", sampler=sampler)
optuna_cbrm.optimize(objective, n_trials=50)

[I 2023-12-10 08:32:46,827] A new study created in memory with name: no-name-c9a3399e-6df2-4c01-b093-c1f822e2d389
<ipython-input-24-2344d44a46cd>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-24-2344d44a46cd>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-24-2344d44a46cd>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample',0,1),
<ipython-input-24-2344d44a46cd>:

In [27]:
cbrm_trial = optuna_cbrm.best_trial
cbrm_trial_params = cbrm_trial.params

cbrm_trial_params

{'iterations': 11983,
 'od_wait': 1321,
 'learning_rate': 0.1838260009078353,
 'reg_lambda': 22.462325635383923,
 'subsample': 0.6313453395998727,
 'random_strength': 32.244184495954144,
 'depth': 12,
 'min_data_in_leaf': 14,
 'leaf_estimation_iterations': 15,
 'bagging_temperature': 2.823194507774442,
 'colsample_bylevel': 0.7113609295575726}

In [29]:
best_model = CatBoostClassifier(**cbrm_trial_params, verbose=False)
best_model.fit(X_train, y_train)
pred_val_cbclf=best_model.predict(X_val)
evulation_metric_val_cb = evulation_metric(y_val, pred_val_cbclf)
evulation_metric_val_cb

0.621273166800967

### test 확인

In [30]:
pred_test_cb=best_model.predict(X_test)

evulation_metric_test_cb=evulation_metric(y_test, pred_test_cb)
print(evulation_metric_test_cb)

0.631578947368421


### submission

In [31]:
X_submission=test[second_best]
print(X_submission)
preds=best_model.predict(X_submission)
preds

      average_time_per_learning_session  recent_learning_achievement
0                              7.047243                    82.074557
1                             24.583327                    80.546040
2                              1.707616                    73.676957
3                             21.391389                    63.314635
4                             18.126987                    55.899647
...                                 ...                          ...
9995                          90.251023                    70.552157
9996                          10.205804                    67.558845
9997                          20.346781                    89.294007
9998                         156.521711                    82.218390
9999                          53.633453                    72.274352

[10000 rows x 2 columns]


array([1, 1, 1, ..., 1, 1, 1])

In [32]:
submission['target']=preds
submission

,user_id,target
0,0001d6e9,1
1,0002c77d,1
2,0002df5b,1
3,000b6068,0
4,00184a0c,0
...,...,...
9995,ffe2eba5,1
9996,ffe710f1,0
9997,ffeccdef,1
9998,fff3fcea,1


In [33]:
unique, counts = np.unique(preds, return_counts = True)
cnt_dict = dict(zip(unique, counts))
cnt_dict

{0: 3610, 1: 6390}

In [34]:
submission.to_csv(base_path+"submission_1210_4.csv", index=False)